In [2]:
import simpy
import random
import pandas as pd
print(f'STARTING SIMULATION')
print(f'----------------------------------')

#-------------------------------------------------

#Parameters

#working hours
hours = 8
#business days
days = 5

#total working time (hours)
total_time = hours * days

#containers
    #plastic
plastic_capacity = 1000
initial_plastic = 500
    #electronic
electronic_capacity = 100
initial_electronic = 100
    #paint
first_assembler_body_capacity = 100
second_assembler_body_capacity = 200 
first_assembler_helice_capacity = 100
second_assembler_helice_capacity = 200 
    #dispatch
dispatch_capacity = 500

machine_availability_body = 0.92
machine_availability_helice = 0.92
tempo_conserto_body = 2
tempo_conserto_helice = 2
total_falhas_body = 0
total_falhas_helice = 0

#-------------------------------------------------

class Drone_Factory:
    def __init__(self, env):
        self.plastic = simpy.Container(env, capacity = plastic_capacity, init = initial_plastic)
        self.electronic = simpy.Container(env, capacity = electronic_capacity, init = initial_electronic)
        self.first_assembler_body = simpy.Container(env, capacity = first_assembler_body_capacity, init = 0)
        self.first_assembler_helice = simpy.Container(env, capacity = first_assembler_helice_capacity, init = 0)
        self.second_assembler_body = simpy.Container(env, capacity = second_assembler_body_capacity, init = 0)
        self.second_assembler_helice = simpy.Container(env, capacity = second_assembler_helice_capacity, init = 0)
        self.dispatch = simpy.Container(env ,capacity = dispatch_capacity, init = 0)
                
def body_maker(env, drone_factory):
    global total_falhas_body  # Declare the global variable to track total body failures

    falhas_corpo = 0  # Variável para rastrear as falhas específicas deste processo
    falha_ocorrida_body = False  # Variável para controlar se a falha ocorreu ou não
    tempo_atual_body = 0  # Variável para rastrear o tempo atual
    
    while True:
        # Verifica se ocorre uma falha
        if not falha_ocorrida_body and random.random() > machine_availability_body:
            falha_ocorrida_body = True
            falhas_corpo += 1  # Incrementa o contador de falhas para corpos
            total_falhas_body += 1  # Incrementa o contador global de falhas
            yield env.timeout(tempo_conserto_body)  # Tempo de conserto
            falha_ocorrida_body = False  # Reinicia a variável após o conserto
            
        else:
            yield drone_factory.plastic.get(1)
            body_time = 2
            yield env.timeout(body_time)
            yield drone_factory.first_assembler_body.put(1)
            tempo_atual_body += body_time
            
            
# Declare a global variable to track total helice failures
total_falhas_helice = 0

# ...

def helice_maker(env, drone_factory):
    global total_falhas_helice  # Declare the global variable to track total helice failures

    falhas_helice = 0  # Variável para rastrear as falhas específicas deste processo
    falha_ocorrida_helice = False  # Variável para controlar se a falha ocorreu ou não
    tempo_atual_helice = 0  # Variável para rastrear o tempo atual

    while True:
         # Verifica se ocorre uma falha
        if not falha_ocorrida_helice and random.random() > machine_availability_helice:
            falha_ocorrida_helice = True
            falhas_helice += 1  # Incrementa o contador de falhas para helices
            total_falhas_helice += 1  # Incrementa o contador global de falhas
            yield env.timeout(tempo_conserto_helice)  # Tempo de conserto
            falha_ocorrida_helice = False  # Reinicia a variável após o conserto
            
        yield drone_factory.plastic.get(1)
        helice_time = 2
        yield env.timeout(helice_time)
        yield drone_factory.first_assembler_helice.put(4)
        tempo_atual_helice += helice_time
        
def painter(env, drone_factory):
    while True:
        yield drone_factory.first_assembler_body.get(2)
        yield drone_factory.first_assembler_helice.get(8)
        paint_time = 1.3
        yield env.timeout(paint_time)
        yield drone_factory.second_assembler_helice.put(8)
        yield drone_factory.second_assembler_body.put(2)

def assembler(env, drone_factory):
    while True:
        yield drone_factory.second_assembler_helice.get(4)
        yield drone_factory.second_assembler_body.get(1)
        yield drone_factory.electronic.get(1)
        assembling_time = 1.1
        yield env.timeout(assembling_time)
        yield drone_factory.dispatch.put(1)

#-------------------------------------------------
        

env = simpy.Environment()
drone_factory = Drone_Factory(env)

body_maker_process = env.process(body_maker(env, drone_factory))
helice_maker_process = env.process(helice_maker(env, drone_factory))
painter_process = env.process(painter(env, drone_factory))
assembler_process = env.process(assembler(env, drone_factory))

env.run(until = total_time)


# Parameters

# Number of simulations
num_simulations = 100

# Define a function to run the simulation
def run_simulation():
    # Create the simulation environment
    env = simpy.Environment()

    # Create an instance of Drone_Factory
    drone_factory = Drone_Factory(env)

    # Run the simulation
    body_maker_process = env.process(body_maker(env, drone_factory))
    helice_maker_process = env.process(helice_maker(env, drone_factory))
    painter_process = env.process(painter(env, drone_factory))
    assembler_process = env.process(assembler(env, drone_factory))

    env.run(until=total_time)

    # Calculate total failures
    total_failures = total_falhas_body + total_falhas_helice

    # Calculate total failures for bodies and helices separately
    total_failures_bodies = total_falhas_body
    total_failures_helices = total_falhas_helice

    # Return the simulation results as a dictionary
    return {
        'Bodies Pré Paint': drone_factory.first_assembler_body.level,
        'Bodies Post Paint': drone_factory.second_assembler_body.level,
        'Propellers Pré Paint': drone_factory.first_assembler_helice.level,
        'Propellers Post Paint': drone_factory.second_assembler_helice.level,
        'Total Drone Production': drone_factory.dispatch.level,
        'Total Failures': total_failures,
        'Total Failures Bodies': total_failures_bodies,
        'Total Failures Helices': total_failures_helices
    }

# Initialize a list to store simulation results
simulation_results = []

# Run the simulation multiple times
for simulation_number in range(num_simulations):
    # Reset failure variables before each simulation
    total_falhas_body = 0
    total_falhas_helice = 0

    # Run the simulation and append the results to the list
    simulation_result = run_simulation()
    simulation_results.append(simulation_result)

# Create a DataFrame from the simulation results
df = pd.DataFrame(simulation_results)

# Print the DataFrame
print(df)


print('medias da simulacao' )
# Calculate the means of simulation metrics
      
mean_metrics = {
    'Mean Bodies Pré Paint': df['Bodies Pré Paint'].mean(),
    'Mean Bodies Post Paint': df['Bodies Post Paint'].mean(),
    'Mean Propellers Pré Paint': df['Propellers Pré Paint'].mean(),
    'Mean Propellers Post Paint': df['Propellers Post Paint'].mean(),
    'Mean Total Drone Production': df['Total Drone Production'].mean(),
    'Mean Total Failures': df['Total Failures'].mean(),
    'Mean Total Failures Bodies': df['Total Failures Bodies'].mean(),
    'Mean Total Failures Helices': df['Total Failures Helices'].mean()
}

# Create a DataFrame for the means
df_means = pd.DataFrame([mean_metrics])

# Print the DataFrame with means
print(df_means)

STARTING SIMULATION
----------------------------------
    Bodies Pré Paint  Bodies Post Paint  Propellers Pré Paint  \
0                  1                  0                     4   
1                  0                  1                     8   
2                  0                  1                     0   
3                  0                  1                     0   
4                  1                  0                     8   
..               ...                ...                   ...   
95                 0                  1                     0   
96                 0                  0                     4   
97                 1                  1                     0   
98                 0                  1                     0   
99                 0                  1                     4   

    Propellers Post Paint  Total Drone Production  Total Failures  \
0                       0                      16               4   
1                       4 